# Introducción al uso de interfaces de datos (API) (II)

## Ejercicio: Consulta multiple a OpenMeteo

En este ejercicio vamos a trabajar en la consulta de varias series temporales en diferentes coordenadas para su posterior análisis y visualización conjunta. En primer lugar generamos un diccionario  de coordenadas de Icod de Los Vinos, Adeje, y Santa Cruz de Tenerife para realizar consultas a la API de OpenMeteo:

In [ ]:
# Generamos un doccionario
coordenadas = {
    'Icod': {
        'latitud': 28.367,
        'longitud': -16.721
    },
    'Adeje': {
        'latitud': 28.1227,
        'longitud': -16.726
    },
    'SantaCruz': {
        'latitud': 28.4636,
        'longitud': -16.2518
    }
}

2. Consultamos la serie de temperatura para cada ubicación y combinamos todas las respuestas en un solo archivo JSON:

In [ ]:
import requests
import json

# Diccionario para almacenar los resultados combinados
datos_combinados = {}

# Bucle para consultar la API para cada localidad
for lugar, coords in coordenadas.items():
    # Construir la URL para la API de OpenMeteo
    url = f"https://api.open-meteo.com/v1/forecast?latitude={coords['latitud']}&longitude={coords['longitud']}&hourly=temperature_2m"
    
    # Realizar la solicitud GET a la API
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Extraer los datos en formato JSON
        data = response.json()
        
        # Almacenar los datos en el diccionario con la localidad como clave
        datos_combinados[lugar] = data
        print(f"Datos obtenidos para {lugar}")
    else:
        print(f"Error al obtener datos para {lugar}: {response.status_code}")


El resultado tendrá un formato como el siguiente:
```json
{
    "Icod": {
        "hourly": {
            "time": ["2024-01-01T00:00", "2024-01-01T01:00", ...],
            "temperature_2m": [12.5, 12.0, ...]
        }
    },
    "Adeje": {
        "hourly": {
            "time": ["2024-01-01T00:00", "2024-01-01T01:00", ...],
            "temperature_2m": [13.5, 13.0, ...]
        }
    },
    "SantaCruz": {
        "hourly": {
            "time": ["2024-01-01T00:00", "2024-01-01T01:00", ...],
            "temperature_2m": [15.0, 14.5, ...]
        }
    }
}



3. Guardamos todos los resultados en un archivo JSON combinado

```python
# Guardar todos los resultados en un archivo JSON combinado
with open('data/m2_e2_datos_meteo_combinados.json', 'w') as json_file:
    json.dump(datos_combinados, json_file, indent=4)

print("Datos combinados guardados en 'temperaturas_combinadas.json'.")
```

### Lectura de datos combinados en formato JSON
Leemos los datos desde el archivo JSON generado previamente y convertimos cada conjunto de datos de temperatura en un DataFrame separado para cada localidad:

In [ ]:
import json
import pandas as pd

# Leer el archivo JSON que contiene los datos combinados
with open('data/m2_e2_datos_meteo_combinados.json', 'r') as json_file:
    datos_combinados = json.load(json_file)

# Diccionario para almacenar los DataFrames de cada localidad
dfs = {}

# Convertir los datos de cada localidad en un DataFrame
for localidad, datos in datos_combinados.items():
    # Crear un DataFrame a partir de los datos de la localidad
    localidad_df = pd.DataFrame({
        'Fecha': pd.to_datetime(datos['hourly']['time']),  # Convertir las fechas a datetime
        'Temperatura (°C)': datos['hourly']['temperature_2m']
    })
    
    # Almacenar el DataFrame en el diccionario usando la localidad como clave
    dfs[localidad] = localidad_df

    # Mostrar el DataFrame de la localidad
    print(f"DataFrame de {localidad}:")
    print(localidad_df.head(), "\n")

# Ahora tienes un DataFrame separado para cada localidad en el diccionario 'dfs'


### Visulización básica de los datos


#### Gráfico de línea
Visualizamos la serie temporal de temperaturas en un gráfico de línea, añadiendo una línea discontinua que represente el promedio de la serie 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

for localidad in dfs.keys():
    df = dfs[localidad]
    # Crear el gráfico de líneas
    plt.figure(figsize=(10, 6))
    plt.plot(df['Fecha'], df['Temperatura (°C)'], linestyle='-', marker='o', markersize=3)

    # Calcular el promedio, máximo y mínimo de la columna 'Temperatura (°C)'
    promedio_temperatura = df['Temperatura (°C)'].mean()
    # Añadir línea discontinua para el promedio
    plt.axhline(y=promedio_temperatura, color='red', linestyle='--', linewidth=2, label=f'Promedio: {promedio_temperatura:.2f} °C')

    # Añadir etiquetas y título
    plt.title('Serie Temporal de Temperatura en {}'.format(localidad), fontsize=14)
    plt.xlabel('Fecha', fontsize=12)
    plt.ylabel('Temperatura (°C)', fontsize=12)

    # Rotar las etiquetas del eje X para mejor legibilidad
    plt.xticks(rotation=45)

    # Mostrar una cuadrícula
    plt.grid(True)

    # Ajustar los márgenes
    plt.tight_layout()

    # Mostrar el gráfico
    plt.show()


#### Histograma
Generamos un histograma de la serie de temperaturas, que nos permitirá visualizar la distribución de las temperaturas en la serie temporal:

In [ ]:
for localidad in dfs.keys():
    df = dfs[localidad]
    # Crear el histograma de la columna 'Temperatura (°C)'
    plt.figure(figsize=(8, 6))
    plt.hist(df['Temperatura (°C)'], bins=30, color='skyblue', edgecolor='black')

    # Añadir etiquetas y título
    plt.title('Histograma de Temperaturas en {}'.format(localidad), fontsize=16)
    plt.xlabel('Temperatura (°C)', fontsize=12)
    plt.ylabel('Frecuencia', fontsize=12)

    # Mostrar el histograma
    plt.grid(True)
    plt.tight_layout()
    plt.show()


### Combinacion de datos
A continuación, combinamos los DataFrames de las tres localidades en un solo DataFrame.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Crear un DataFrame combinado a partir de los DataFrames individuales
combinado_df = pd.DataFrame()

# Agregar cada DataFrame de localidad al DataFrame combinado
for localidad, localidad_df in dfs.items():
    # Renombrar la columna de temperatura para incluir el nombre de la localidad
    localidad_df_tmp = localidad_df.rename(columns={'Temperatura (°C)': f'Temperatura ({localidad})'})
    
    # Si es el primer DataFrame, se inicia el DataFrame combinado con las fechas
    if combinado_df.empty:
        combinado_df['Fecha'] = localidad_df_tmp['Fecha']
    
    # Agregar la columna de temperaturas de la localidad
    combinado_df[f'Temperatura ({localidad})'] = localidad_df_tmp[f'Temperatura ({localidad})']

# Mostrar el DataFrame combinado
print(combinado_df.head())

### Visualizaciñon combinada series temporales
Visualizamos las tres series de temperatura en un gráfico de líneas manteniendo el promedio de cada una y, finalmente, guarda el DataFrame combinado en un archivo CSV:

In [ ]:
# Visualizar las 3 series de temperatura en un solo gráfico con el promedio de cada serie

plt.figure(figsize=(10, 6))

# Colores y etiquetas de cada localidad
colors = {'Icod': 'blue', 'Adeje': 'green', 'SantaCruz': 'red'}

# Dibujar cada serie de temperaturas y sus promedios
for localidad, color in colors.items():
    # Graficar la serie de cada localidad
    plt.plot(combinado_df['Fecha'], combinado_df[f'Temperatura ({localidad})'], color=color, linestyle='-', marker='o', markersize=3, label=f'Temperatura ({localidad})')
    
    # Calcular el promedio de la temperatura de la localidad
    promedio = combinado_df[f'Temperatura ({localidad})'].mean()
    
    # Dibujar una línea horizontal para el promedio de cada localidad
    plt.axhline(y=promedio, color=color, linestyle='--', linewidth=2, label=f'Promedio {localidad}: {promedio:.2f} °C')

# Añadir etiquetas y título
plt.title('Serie Temporal de Temperaturas en Icod de Los Vinos, Adeje y Santa Cruz de Tenerife', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Temperatura (°C)', fontsize=12)

# Rotar las etiquetas del eje X para mejor legibilidad
plt.xticks(rotation=45)

# Mostrar la cuadrícula
plt.grid(True)

# Mostrar la leyenda
plt.legend()

# Ajustar los márgenes
plt.tight_layout()

# Mostrar el gráfico
plt.show()

# Guardar el DataFrame combinado en un archivo CSV
combinado_df.to_csv('data/m2_e2_datos_meteo_combinados.csv', index=False)
print("DataFrame combinado guardado en 'temperaturas_combinadas_localidades.csv'.")
